## Initialization

In [1]:
struct Siddon{ArrFloat, ArrInt}
    origin::ArrFloat
    target::ArrFloat
    spacing::ArrFloat
    isocenter::ArrFloat
    dims::ArrInt
end

In [2]:
function get_α(i::Int64, j::Int64; sid::Siddon)
    planes = [i, j]
    return @. (sid.isocenter + planes * sid.spacing - sid.origin) / (sid.target - sid.origin)
end


function get_α_minmax(sid::Siddon)    
    αx0, αy0 = get_α(0, 0; sid)
    αx1, αy1 = sid.dims .- 1 |> nxy -> get_α(nxy...; sid)
    αxmin, αxmax = minmax(αx0, αx1)
    αymin, αymax = minmax(αy0, αy1)
    αmin = max(αxmin, αymin)
    αmax = min(αxmax, αymax)
    return αxmin, αxmax, αymin, αymax, αmin, αmax
end

get_α_minmax (generic function with 1 method)

In [3]:
function get_φ(α::Float64; sid::Siddon)
    pxyz = @. sid.origin + α * (sid.target - sid.origin)
    return @. (pxyz - sid.isocenter) / sid.spacing
end

get_φ (generic function with 1 method)

In [4]:
function get_idx_minmax(
    αmin::Float64, αmax::Float64, αxmin::Float64, αxmax::Float64,
    ixmin::Float64, ixmax::Float64, p1::Float64, p2::Float64, nx::Int64
)
    if p1 ≤ p2
        imin = αmin == αxmin ? 1 : trunc(Int, ixmin + 1)
        imax = αmax == αxmax ? nx - 1 : trunc(Int, ixmax)
    else
        imin = αmax == αxmax ? 1 : trunc(Int, ixmax + 1)
        imax = αmin == αxmin ? nx - 2 : trunc(Int, ixmin)
    end
    return imin, imax
end



function initialize(sid::Siddon)
    αxmin, αxmax, αymin, αymax, αmin, αmax = get_α_minmax(sid)
    ixmin, jxmin = get_φ(αmin; sid)
    ixmax, jxmax = get_φ(αmax; sid)
    imin, imax = get_idx_minmax(αmin, αmax, αxmin, αxmax, ixmin, ixmax, sid.origin[1], sid.target[1], sid.dims[1])
    jmin, jmax = get_idx_minmax(αmin, αmax, αymin, αymax, jxmin, jxmax, sid.origin[2], sid.target[2], sid.dims[1])
    return αmin, αmax, imin, imax, jmin, jmax
end

initialize (generic function with 1 method)

In [5]:
# Little test implementation
ΔX, ΔY = 1, 1
X₀, Y₀ = 0, 0

nx = 5  # Number of planes (x)
ny = 5  # Number of planes (y)

origin = [-0.5, 0.5]
target = [4.5, 4.5]
spacing = [ΔX, ΔY]
isocenter = [X₀, Y₀]
dims = [nx, ny]

sid = Siddon(origin, target, spacing, isocenter, dims)

LoadError: MethodError: no method matching Siddon(::Vector{Float64}, ::Vector{Float64}, ::Vector{Int64}, ::Vector{Int64}, ::Vector{Int64})
[0mClosest candidates are:
[0m  Siddon(::ArrFloat, ::ArrFloat, [91m::ArrFloat[39m, [91m::ArrFloat[39m, ::ArrInt) where {ArrFloat, ArrInt} at In[1]:2

In [6]:
# Test the initialization function
αmin, αmax, imin, _, jmin, _ = initialize(sid)

LoadError: UndefVarError: sid not defined

## Loop

In [7]:
CartesianIndex(trunc.(Int, get_φ(0.1; sid))...)

LoadError: UndefVarError: sid not defined

In [8]:
get_vox_idx(α::Float64) = trunc.(Int, get_φ(α; sid)) .+ 1

get_vox_idx (generic function with 1 method)

In [9]:
function get_voxel_idx(α::Float64; sid::Siddon)
    xidx = get_φ(α; sid)
    idxs = trunc.(Int, xidxs) .+ 1
    return CartesianIndex(idxs...)
end

get_voxel_idx (generic function with 1 method)

In [10]:
iu = sid.origin[1] ≤ sid.target[1] ? 1 : -1
ju = sid.origin[2] ≤ sid.target[2] ? 1 : -1

update_i = [iu, ju]
update_α = @. sid.spacing / abs(sid.target-sid.origin)

αcurr = αmin

αs = get_α(imin, jmin; sid)
αnext, idx = findmin(αs)

αmid = (αnext + αcurr) / 2
voxs = get_vox_idx(αmid)

LoadError: UndefVarError: sid not defined

In [11]:
len = αnext - αcurr
@show αnext, len, voxs

while αcurr < αmax
    voxs[idx] += update_i[idx]
    αs[idx] += update_α[idx]
    
    αnext, idx = findmin(αs)
    
    len = αnext - αcurr
    @show αnext, len, voxs
    
    αcurr = αnext
end

LoadError: UndefVarError: αnext not defined

## 3D?

In [12]:
function get_α(i::Int, j::Int, k::Int; sid::Siddon)
    planes = [i, j, k]
    return @. (sid.isocenter + planes * sid.spacing - sid.origin) / (sid.target - sid.origin)
end


function get_α_minmax(sid::Siddon)
    αx0, αy0, αz0 = get_α(0, 0, 0; sid)
    αx1, αy1, αz1 = (sid.dims .- 1) |> nxyz -> get_α(nxyz...; sid)
    αxmin, αxmax = minmax(αx0, αx1)
    αymin, αymax = minmax(αy0, αy1)
    αzmin, αzmax = minmax(αz0, αz1)
    αmin = max(αxmin, αymin, αzmin)
    αmax = min(αxmax, αymax, αzmax)
    return αxmin, αxmax, αymin, αymax, αzmin, αzmax, αmin, αmax
end


function get_φ(α::Float64; sid::Siddon)
    pxyz = @. sid.origin + α * (sid.target - sid.origin)  # Trace the ray
    return @. (pxyz - sid.isocenter) / sid.spacing
end


function get_idx_minmax(
    αmin::Float64, αmax::Float64, αxmin::Float64, αxmax::Float64,
    ixmin::Float64, ixmax::Float64, p1::Float64, p2::Float64, nx::Int64
)
    if p1 ≤ p2
        imin = αmin == αxmin ? 1 : trunc(Int, ixmin + 1)
        imax = αmax == αxmax ? nx - 1 : trunc(Int, ixmax)
    else
        imin = αmax == αxmax ? 1 : trunc(Int, ixmax + 1)
        imax = αmin == αxmin ? nx - 2 : trunc(Int, ixmin)
    end
    return imin, imax
end


function initialize(sid::Siddon)
    αxmin, αxmax, αymin, αymax, αzmin, αzmax, αmin, αmax = get_α_minmax(sid)
    ixmin, jxmin, kxmin = get_φ(αmin; sid)
    ixmax, jxmax, kxmax = get_φ(αmax; sid)
    imin, imax = get_idx_minmax(αmin, αmax, αxmin, αxmax, ixmin, ixmax, sid.origin[1], sid.target[1], sid.dims[1])
    jmin, jmax = get_idx_minmax(αmin, αmax, αymin, αymax, jxmin, jxmax, sid.origin[2], sid.target[2], sid.dims[2])
    kmin, kmax = get_idx_minmax(αmin, αmax, αzmin, αzmax, kxmin, kxmax, sid.origin[3], sid.target[3], sid.dims[3])
    return αmin, αmax, imin, imax, jmin, jmax, kmin, kmax
end


# TODO: When to convert to CartesianIndex?
function get_voxel_idx(α::Float64; sid::Siddon)
    xidxs = get_φ(α; sid)
    idxs = trunc.(Int, xidxs) .+ 1
    return idxs
    # return CartesianIndex(idxs...)
end


# Main function to evaluate
function (sid::Siddon)()

    # Get idx update conditions
    iu = sid.origin[1] ≤ sid.target[1] ? 1 : -1
    ju = sid.origin[2] ≤ sid.target[2] ? 1 : -1
    ku = sid.origin[3] ≤ sid.target[3] ? 1 : -1
    update_idx = [iu, ju, ku]

    # Get α update conditions
    update_α = @. sid.spacing / abs(sid.target - sid.origin)

    # Initialize the loop
    αmin, αmax, imin, imax, jmin, jmax, kmin, kmax = initialize(sid)

    αcurr = αmin
    steps = get_α(imin, jmin, kmin; sid)
    αnext, idx = findmin(steps)
    αmid = (αcurr + αnext) / 2
    voxel = get_voxel_idx(αmid; sid)

    while αcurr < αmax
        voxel[idx] += update_idx[idx]
        steps[idx] += update_α[idx]
        αnext, idx = findmin(steps)
        step_len = αnext - αcurr
        @show αnext, step_len, voxel
        αcurr = αnext
    end

end

In [13]:
# Little test implementation
ΔX, ΔY, ΔZ = 1, 1, 1
X₀, Y₀, Z₀ = 0, 0, 0

nx = 5  # Number of planes (x)
ny = 5  # Number of planes (y)
nz = 5


origin = [-0.5, 0.5, -0.5]
target = [4.5, 4.5, 4.5]
spacing = Float64[ΔX, ΔY, ΔZ]
isocenter = Float64[X₀, Y₀, Z₀]
dims = [nx, ny, nz]

sid = Siddon(origin, target, spacing, isocenter, dims)

Siddon{Vector{Float64}, Vector{Int64}}([-0.5, 0.5, -0.5], [4.5, 4.5, 4.5], [1.0, 1.0, 1.0], [0.0, 0.0, 0.0], [5, 5, 5])

In [14]:
sid()

(αnext, step_len, voxel) = (0.3, 0.19999999999999998, [1, 2, 1])
(αnext, step_len, voxel) = (0.3, 0.0, [2, 2, 1])
(αnext, step_len, voxel) = (0.375, 0.07500000000000001, [2, 2, 2])
(αnext, step_len, voxel) = (0.5, 0.125, [2, 3, 2])
(αnext, step_len, voxel) = (0.5, 0.0, [3, 3, 2])
(αnext, step_len, voxel) = (0.625, 0.125, [3, 3, 3])
(αnext, step_len, voxel) = (0.7, 0.07499999999999996, [3, 4, 3])
(αnext, step_len, voxel) = (0.7, 0.0, [4, 4, 3])
(αnext, step_len, voxel) = (0.875, 0.17500000000000004, [4, 4, 4])
